## Data loading

First we need to load the MNIST dataset from disk. Since in this exercise we are doing binary classification, i.e. classification between two classes, we only pick the digits 1 and 8 from the MNIST dataset here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import edf
import mnist_loader
from skimage.transform import resize
np.random.seed(1234)

x_train, y_train = mnist_loader.load_mnist(section = 'training', path = 'MNIST')
x_test, y_test = mnist_loader.load_mnist(section = 'testing', path = 'MNIST')

digits = [1, 8]
train_subset_mask = np.logical_or(y_train == digits[0], y_train == digits[1])
x_train = x_train[train_subset_mask]
y_train = y_train[train_subset_mask]
y_train[y_train == digits[0]] = 0
y_train[y_train == digits[1]] = 1
test_subset_mask = np.logical_or(y_test == digits[0], y_test == digits[1])
x_test = x_test[test_subset_mask]
y_test = y_test[test_subset_mask]
y_test[y_test == digits[0]] = 0
y_test[y_test == digits[1]] = 1

plt.imshow(x_train[0], cmap='gray', interpolation = 'nearest')
plt.show()
plt.imshow(x_train[600], cmap='gray', interpolation = 'nearest')
plt.show()

## Creating downsampled versions of the dataset

In the end of this exercise we explore how the classification accuracy depends on the input image resolution. The code below creates 4 different versions of the dataset. This takes some time to execute. During your development you only need to execute this cell and the above one once and then you can modify and test your implementations below with the dataset already beeing loaded. (big advantage of jupyter notebook)

In [ ]:
def resize_images(images, res_x, res_y):
    resized_images = np.zeros((images.shape[0], res_x, res_y))
    for n in range(len(images)):
        resized_images[n, :, :] = resize(images[n, :, :], resized_images.shape[1:], anti_aliasing=False)
    return resized_images

# The 28x28 images are flattend into a feature (=input) vector of size 784 here
x_train_full_res = x_train.reshape(x_train.shape[0], -1) # flatten
x_test_full_res = x_test.reshape(x_test.shape[0], -1) # flatten
print(x_train_full_res.shape, x_test_full_res.shape)

# The means of the flattend images is taken. So in this case we only have a single input feature
x_train_mean = np.mean(x_train_full_res, axis=1, keepdims=True)
x_test_mean = np.mean(x_test_full_res, axis=1, keepdims=True)
print(x_train_mean.shape, x_test_mean.shape)

# 28x28 images are downscaled into 4x4 images and then flattened to arrive at 16 features
x_train_4x4 = resize_images(x_train, 4, 4).reshape(x_train.shape[0], -1)
x_test_4x4 = resize_images(x_test, 4, 4).reshape(x_test.shape[0], -1)
print(x_train_4x4.shape, x_test_4x4.shape)

# 28x28 images are downscaled into 8x8 images and then flattened to arrive at 64 features
x_train_8x8 = resize_images(x_train, 8, 8).reshape(x_train.shape[0], -1)
x_test_8x8 = resize_images(x_test, 8, 8).reshape(x_test.shape[0], -1)
print(x_train_8x8.shape, x_test_8x8.shape)

## Logistic regression based on hand derived derivative
Implement the function `compute_derivative`. It often helps to print the `shape` of numpy arrays.

In [ ]:
def compute_derivative(x, y, y_hat):
    pass

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def Xavier(num_features):
    return np.sqrt(3.0 / num_features)

def analytical_train_and_test(num_epochs, batch_size, learning_rate, x_train, y_train, x_test, y_test):
    # initalize our parameters w randomly
    num_features = x_train.shape[1]
    xavier_init = Xavier(num_features) # 
    w = np.random.uniform(-xavier_init, xavier_init, size=(num_features + 1,))
    w[-1] = 0 # set bias term to zero
    
    train_err_log = []
    test_err_log = []
    for epoch in range(num_epochs):
        print("Epoch: {}/{}".format(epoch + 1, num_epochs))
        train_err = analytical_run_epoch(batch_size, x_train, y_train, w, 'train', learning_rate=learning_rate)
        train_err_log.append(train_err)
        print ("\t Training Error {:.2f} %".format(train_err))
        test_err = analytical_run_epoch(x_test.shape[0], x_test, y_test, w, 'test')
        test_err_log.append(test_err)
        print ("\t Test Error {:.2f} %".format(test_err))
    return train_err_log, test_err_log
    
def analytical_run_epoch(batch_size, x, y, w, phase, learning_rate=None):
    dataset_size = len(x)
    total_err = 0.0
    num_batches = dataset_size // batch_size
    for i in range(num_batches):
        start, end = i * batch_size, (i + 1) * batch_size
        x_batch = x[start:end]
        y_batch = y[start:end]
        one_column_vector = np.ones((x_batch.shape[0], 1)) # a column vector with all entries beeing 1
        x_batch = np.concatenate((x_batch, one_column_vector), axis=1) # append this column vector to x
        y_hat = sigmoid(x_batch.dot(w))
        if phase == 'train':
            w_derivative = compute_derivative(x_batch, y_batch, y_hat)
            w[:] = w - learning_rate * w_derivative
        prediction = (y_hat > 0.5)
        total_err += np.sum(np.not_equal(prediction, y_batch))
    return 100 * total_err / dataset_size

def plot(train_err_log, test_err_log):
    plt.xlabel("epochs")
    plt.ylabel("error (%)")
    plt.plot(np.arange(len(test_err_log)), test_err_log, color='red')
    plt.plot(np.arange(len(train_err_log)), train_err_log, color='blue')
    plt.legend(['test error', 'train error'], loc='upper right')
    plt.show()
    plt.clf()

In [ ]:
num_epochs = 10
batch_size = 64
learning_rate = 0.5
train_err_log, test_err_log = analytical_train_and_test(num_epochs, batch_size, learning_rate,
                                             x_train_mean, y_train, x_test_mean, y_test)
plot(train_err_log, test_err_log)

## Exercises based on EDF

Implement the `forward` and `backward` method of this class. Take a look at `edf.py` for inspiration.

In [ ]:
# input shape: (batch size, 1)
# output shape: (batch size, 2)
class SingleProbToProbVector(edf.CompNode):
    def __init__(self, z):
        edf.CompNodes.append(self)
        self.z = z

    def forward(self):
        pass

    def backward(self):
        pass

### Logistic Regression based on EDF and mean images

Use EDF to assemble a computational graph for logistic regression with the "mean" images as input. You will need the `SingleProbToProbVector` node that you implemented above. The function `train_and_test` below expects variables `x_node`, `y_node`, `prob_node` and `loss_node` to be defined. `prob_node` calculates the class probabilities for each training sample, while the `loss_node` calculates the loss for the entire training batch.

In [ ]:
edf.clear_compgraph()
# Define the computation graph here

In [ ]:
"""the following functions are used to train the network.
training is done by iterating over mini-batches of size 'batch_size'
and updating the model's parameters with SGD"""

def run_epoch(batch_size, x, y, x_node, y_node, prob_node, loss_node=None):
    dataset_size = x.shape[0]
    total_err = 0.0
    num_batches = dataset_size // batch_size
    for i in range(num_batches):
        start, end = i * batch_size, (i + 1) * batch_size
        x_node.value = x[start:end]
        y_node.value = y[start:end]
        edf.Forward()
        total_err += np.sum(np.not_equal(np.argmax(prob_node.value, axis=1), y_node.value))
        if loss_node:
            edf.Backward(loss_node)
            edf.UpdateParameters()
    return 100 * total_err / dataset_size

def train_and_test(num_epochs, batch_size, x_train, y_train, x_test, y_test,
                   x_node, y_node, prob_node, loss_node):
    train_err_log = []
    test_err_log = []
    for epoch in range(num_epochs):
        print("Epoch: {}/{}".format(epoch + 1, num_epochs))
        train_err = run_epoch(batch_size, x_train, y_train, x_node, y_node, prob_node, loss_node)
        train_err_log.append(train_err)
        print ("\t Training Error {:.2f} %".format(train_err))
        test_err = run_epoch(len(x_test), x_test, y_test, x_node, y_node, prob_node)
        test_err_log.append(test_err)
        print ("\t Test Error {:.2f} %".format(test_err))
    return train_err_log, test_err_log

In [ ]:
"""now, we are ready to train the network. we can choose SGD's learning rate
by changing edf.learning_rate, which we will set as 0.5 for now."""

num_epochs = 10
batch_size = 64
edf.learning_rate = 0.5
train_err_log, test_err_log = train_and_test(num_epochs, batch_size, x_train_mean, y_train, x_test_mean, y_test,
                                             x_node, y_node, prob_node, loss_node)
plot(train_err_log, test_err_log)

## 4x4, 8x8 and full resolution experiments based on EDF

Define computational graphs for the 4x4, 8x8 and full resolution experiments and run them. You need to call `edf.clear_compgraph()` before defining a new computation graph.

### 4x4 experiment

### 8x8 experiment

### full res (=28x28) experiment